# 14 privocy

### メモ  
GAOZで完全一致しないやつがある  
->ハミング距離最小のやつで絞る

### 担当者と日付
2024/10/1 渡邉健  
2024/10/2 ごとう 渡邉

## 準備

### ライブラリのインポート  

In [50]:
import pandas as pd
import numpy as np
import itertools
import math
import warnings #ヘッダとかのやつが邪魔だったのでファイルにまとめてインポートしてる
warnings.simplefilter(action='ignore', category=FutureWarning)

In [51]:
# fmt: off
ALL_HEADER = ['Name', 'Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '247', '260', '653', '673', '810', '885',  # noqa: E501
                      '1009', '1073', '1097', '1126', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017',  # noqa: E501
                      '2021', '2043', '2086', '2087', '2093', '2100', '2105', '2138', '2143', '2174', '2193', '2253',  # noqa: E501
                      '2399', '2628', '2797', '2872', '2968', '3393', '3438', '3439', '3440', '3466', '3479', '3489',  # noqa: E501
                      '3877', '3889'] # noqa

B_HEADERS_LIST = [
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '653', '1525', '2105', '2193', '2253', '2628', '2872', '3438', '3439', '3440', '3877', '3889'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '260', '653', '673', '1009', '1073', '1525', '1750', '1881', '1967', '2043', '2093', '2105', '2143', '2193', '2399', '2628', '2968', '3479', '3489', '3877', '3889'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '673', '1881', '1920', '2087', '2138'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '673', '810', '885', '1009', '1073', '1097', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2043', '2087', '2093', '2138', '2399', '3438', '3439', '3440'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '673', '810', '1073', '1126', '1702', '2100', '2174', '2253', '2797', '3393', '3466'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '885', '1097', '1654', '2086', '2138', '2872'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '2100', '2143', '2872', '3479'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '1097', '1750', '2021', '2093', '2105', '2628', '2968'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '1920', '2017', '2087'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '1097', '2628', '2174', '2797', '1073', '2100', '2968', '2105', '2193'],  # noqa: E501
]

B_USER_ATTRIBUTE_HEADERS = ['Gender', 'Age', 'Occupation', 'ZIP-code']

B_REVIEW_HEADERS_LIST = [
  ['260', '653', '1525', '2105', '2193', '2253', '2628', '2872', '3438', '3439', '3440', '3877', '3889'],  # noqa: E501
  ['2', '56', '260', '653', '673', '1009', '1073', '1525', '1750', '1881', '1967', '2043', '2093', '2105', '2143', '2193', '2399', '2628', '2968', '3479', '3489', '3877', '3889'],  # noqa: E501
  ['673', '1881', '1920', '2087', '2138'],  # noqa: E501
  ['2', '56', '673', '810', '885', '1009', '1073', '1097', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2043', '2087', '2093', '2138', '2399', '3438', '3439', '3440'],  # noqa: E501
  ['673', '810', '1073', '1126', '1702', '2100', '2174', '2253', '2797', '3393', '3466'],  # noqa: E501
  ['247', '885', '1097', '1654', '2086', '2138', '2872'],  # noqa: E501
  ['247', '2100', '2143', '2872', '3479'],  # noqa: E501
  ['260', '1097', '1750', '2021', '2093', '2105', '2628', '2968'],  # noqa: E501
  ['247', '1920', '2017', '2087'],  # noqa: E501
  ['260', '1097', '2628', '2174', '2797', '1073', '2100', '2968', '2105', '2193'],  # noqa: E501
]

MOVIE_IDS = ['2', '56', '247', '260', '653', '673', '810', '885', '1009', '1073', '1097', '1126', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2021', '2043', '2086', '2087', '2093', '2100', '2105', '2138', '2143', '2174', '2193', '2253', '2399', '2628', '2797', '2872', '2968', '3393', '3438', '3439', '3440', '3466', '3479', '3489', '3877', '3889']
# fmt: on

### C_0 ~ C_9までの読み込み

In [52]:
from os import listdir

ATTACK_TARGET = "14"

#c0~c9に対するdfのリスト
c_data_list = []

files = listdir("../data")

for i in range(10):
    file = None
    for f in files:
        if f.startswith(f"C{ATTACK_TARGET}_{i}"):
            file = f
            break
    if file is None:
        print(f)
        raise("File not found")
    c_data = pd.read_csv(f"../data/C{ATTACK_TARGET}_{i}.csv")
    c_data_list.append(c_data)

c_data_list[0]

,Gender,Age,Occupation,ZIP-code,260,653,1525,2105,2193,2253,2628,2872,3438,3439,3440,3877,3889
0,F,56,9,230,1,1,1,2,0,5,3,4,2,0,0,2,1
1,F,50,1,902,5,2,2,0,1,4,2,0,1,0,2,1,5
2,M,45,9,945,3,3,0,1,5,0,0,0,3,0,0,0,0
3,M,25,0,830,1,0,0,2,0,0,5,0,0,0,0,0,0
4,F,18,11,522,3,1,0,1,2,0,4,1,0,5,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,F,25,6,503,1,5,4,5,0,5,5,5,5,3,1,3,0
9996,F,50,9,982,2,0,1,2,5,1,4,4,2,3,3,4,4
9997,M,18,17,945,4,1,0,3,3,0,4,4,4,5,1,0,2
9998,M,45,19,988,3,1,3,0,5,2,4,1,5,3,0,1,2


### GAOZの確率変数の生起確立

In [53]:
gen_dist = {}
age_dist = {}
ocp_dist = {}
zip_dist = {}

for cf in c_data_list:
    for i in range(10000):
        gender = cf.iloc[i]['Gender']
        age = f'{cf.iloc[i]['Age']}'
        ocp = f'{cf.iloc[i]['Occupation']}'
        zip = f'{cf.iloc[i]['ZIP-code']}'
        if gender in gen_dist.keys():
            gen_dist[gender] += 1
        else:
            gen_dist[gender] = 1
        if age in age_dist.keys():
            age_dist[age] += 1
        else:
            age_dist[age] = 1
        if ocp in ocp_dist.keys():
            ocp_dist[ocp] += 1
        else:
            ocp_dist[ocp] = 1
        if zip in zip_dist.keys():
            zip_dist[zip] += 1
        else:
            zip_dist[zip] = 1
for key in gen_dist.keys():
    gen_dist[key] /= 100000
for key in age_dist.keys():
    age_dist[key] /= 100000
for key in ocp_dist.keys():
    ocp_dist[key] /= 100000
for key in zip_dist.keys():
    zip_dist[key] /= 100000 
print(gen_dist)
print(age_dist)
print(ocp_dist)
print(zip_dist)

{'F': 0.3943, 'M': 0.6057}
{'56': 0.1076, '50': 0.1091, '45': 0.1401, '25': 0.2126, '18': 0.1816, '35': 0.1675, '1': 0.0815}
{'9': 0.0469, '1': 0.0361, '0': 0.0666, '11': 0.0523, '16': 0.0328, '4': 0.0871, '7': 0.0654, '3': 0.03, '14': 0.0576, '2': 0.062, '20': 0.0453, '5': 0.0529, '8': 0.0426, '13': 0.0282, '17': 0.071, '12': 0.0474, '15': 0.0581, '18': 0.0375, '10': 0.0279, '6': 0.0266, '19': 0.0257}
{'230': 0.0026, '902': 0.0015, '945': 0.0026, '830': 0.0024, '522': 0.0019, '118': 0.0012, '933': 0.0036, '486': 0.0013, '292': 0.0015, '161': 0.0008, '969': 0.0045, '286': 0.0033, '6': 0.0004, '474': 0.0034, '38': 0.0035, '148': 0.0013, '424': 0.0023, '740': 0.0024, '987': 0.0011, '582': 0.0036, '956': 0.0017, '992': 0.0009, '483': 0.0019, '950': 0.002, '78': 0.0021, '10': 0.0024, '750': 0.0039, '12': 0.001, '234': 0.0023, '701': 0.0014, '42': 0.0018, '910': 0.0029, '338': 0.0055, '773': 0.0033, '24': 0.0012, '601': 0.0024, '602': 0.0028, '30': 0.001, '333': 0.0054, '306': 0.003, '380':

### Ba, Bbの読み込み

In [54]:
Ba = pd.read_csv(f"../data/B{ATTACK_TARGET}a.csv", names=ALL_HEADER)
Ba = Ba.drop(0)
Ba.head()

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
1,Dionisio Leile,M,35,4,503,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Zacharias Greenman,M,18,5,231,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Buiron Hartas,M,50,9,322,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Vassily Hugonneau,M,35,9,28,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Emlynne Wernher,F,45,15,731,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
Bb = pd.read_csv(f"../data/B{ATTACK_TARGET}b.csv")
Bb.head()

,2,56,247,260,653,673,810,885,1009,1073,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
0,4,4,0,1,5,4,4,0,3,*,...,1,5,4,0,0,3,5,2,2,2
1,2,0,0,3,3,1,3,2,0,1,...,3,1,3,1,0,5,4,5,5,1
2,4,1,4,1,2,0,0,5,2,2,...,5,4,4,5,1,4,2,2,5,1
3,4,4,3,2,0,0,0,0,*,4,...,0,0,0,3,0,0,5,5,0,2
4,2,0,0,5,1,2,4,3,4,1,...,0,1,5,0,0,4,3,2,3,3


In [56]:
Bals = Ba.values.tolist()
Bbls = Bb.values.tolist()
print(Bals)
print(Bbls)

[['Dionisio Leile', 'M', '35', '4', '503', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], ['Zacharias Greenman', 'M', '18', '5', '231', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], ['Buiron Hartas', 'M', '50', '9', '322', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], ['Vassily Hugonneau', 'M', '35', '9', '28', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan

In [57]:
np.isnan(Bals[0][6])

np.True_

## 分析

### GAOZに関して、BaのN番目の行と完全一致する行をC0~C9から取得　　
-GAOZへの変更が行われているか

In [58]:
# 試しにBaの3行目を選択
N = 3

gender = Ba.iloc[N]["Gender"]
age = int(Ba.iloc[N]["Age"])
occupation = int(Ba.iloc[N]["Occupation"])
zipcode = int(Ba.iloc[N]["ZIP-code"])

print("検索条件")
print(gender, age, occupation, zipcode)

検索条件
M 35 9 28


### GAOZに関して、各Ciの中でHamming distが最小のものをリスト化する
最小のものだけなく、ある程度のバッファがあってもいいのかな  
いまのだと、ZIP-codeが同じかどうかで分類してから残りの３つで類似度評価になってる気がするから、  
hamming distanceの重みづけも最高するべきかな  
GAOZのシャッフルが効果的に行われてるとしたらもはたGender考慮しなくてもいいかもしれない
というか、それぞれの属性の値の匿名化データにおける頻度によって重みづけを考えてもいいかも
確率分布のdict作っといたほうがいいかな
ていうか重みづけで考えるならhamming distance別にfloatでいいな
ハミング距離はどれだけ遠いかを評価する指標
->レアな値を探そうとして違っていていも、大して問題じゃない気がする？いや、逆な気もするけど、いったんこの考えかたで行ってみよう

In [59]:
import copy

def GAOZ_weighted_hamming(comp, i_gender, i_age, i_occupation, i_zipcode):
    ham = 0
    if comp['Gender'] != i_gender:
        ham += 1
    if comp['Age'] != i_age:
        ham += 2
    if  comp['Occupation'] != i_occupation:
        ham += 2
    if comp['ZIP-code'] != i_zipcode:
        ham += 3
    return ham


In [60]:

filtered_C_list = []
for c_index, cf in enumerate(c_data_list):
    min_hamming_distance = float("inf")
    cand = []
    # Cの10000レコードのそれぞれにハミング距離を計算
    for index, c_row in cf.iterrows():
        hamming_distance = GAOZ_weighted_hamming(c_row, gender, age, occupation, zipcode)
        if hamming_distance == min_hamming_distance:
            cand.append((c_row, hamming_distance))
        if hamming_distance < min_hamming_distance:
            min_hamming_distance = hamming_distance
            cand = [(c_row, hamming_distance)]
    # ハミング距離とCのファイルのindexを追加
    for elem, hamming_distance in cand:
        headers = copy.deepcopy(ALL_HEADER)
        headers.append("GAOZ_Hamming_Dist")
        headers.append("C_index")
        elem["GAOZ_Hamming_Dist"] = hamming_distance
        elem["C_index"] = c_index
        filtered_C_list.append(pd.Series(elem, index=headers))

merged_df = pd.DataFrame(filtered_C_list)
merged_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,3438,3439,3440,3466,3479,3489,3877,3889,GAOZ_Hamming_Dist,C_index
669,NaN,M,35,9,28,NaN,NaN,NaN,3.0,0.0,...,3.0,0.0,5.0,NaN,NaN,NaN,0.0,1.0,0,0
3832,NaN,M,35,9,28,4.0,0.0,NaN,5.0,0.0,...,NaN,NaN,NaN,NaN,0.0,3.0,0.0,1.0,0,1
2421,NaN,M,35,9,28,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2
862,NaN,M,35,9,28,4.0,0.0,NaN,NaN,NaN,...,3.0,0.0,5.0,NaN,NaN,NaN,NaN,NaN,0,3
8053,NaN,M,35,9,28,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,4
2659,NaN,M,35,9,28,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,5
1560,NaN,M,35,9,28,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,0,6
9997,NaN,M,35,9,28,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,7
2621,NaN,M,35,9,28,NaN,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,8
8273,NaN,M,35,9,28,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,9


### データの再現　　
各行に対して最頻値を一つ値を取得する

In [61]:
import statistics
from collections import Counter

def select_random_mode(column):
    """
    最頻値を取得
    最頻値が複数ある場合はランダムに1つ選択
    """
    # NaNを除外した値のリストを作成
    valid_values = column.dropna().tolist()
    if not valid_values:
        # 有効な値がない場合はNaNを返す
        return np.nan
    # 最頻値とその出現回数を取得
    value_counts = Counter(valid_values)
    max_count = max(value_counts.values())
    # 最頻値のリストを作成
    modes = [value for value, count in value_counts.items() if count == max_count]
    # 最頻値が1つの場合はそれを返し、複数ある場合はランダムに1つ選択
    return np.random.choice(modes)

def select_random_value(column):
    # NaNを除外した値のリストを作成
    valid_values = column.dropna().tolist()
    if valid_values:
        # 有効な値がある場合、ランダムに1つ選択
        return np.random.choice(valid_values)
    else:
        # 有効な値がない場合はNaNを返す
        return np.nan


おそらくめちゃくちゃ時間がかかることになるけど  
各カラムの値の組み合わせを全部試してBbと比較して  
最小hamming distancが最小のものを組み合わせとするようにしてみる  
->最小のものが複数あった場合の判断基準は?  
->最終的な回答の組み合わせも考慮してもいいかも  
->組み合わせの総数多すぎてカーネルクラッシュする  
->0が候補に上がってるカラムは0で確定しちゃう

In [62]:

# 各カラムからランダムに値を選択
# random_values = {}
# for column in ALL_HEADER:
#     random_values[column] = select_random_value(merged_df[column])

# # 結果を新しいデータフレームとして作成
# result_df = pd.DataFrame([random_values])
# result_df

In [63]:
c_list_list = [data.values.tolist() for data in c_data_list]
c_list_list[1]
Bbls

[['4',
  '4',
  0,
  '1',
  '5',
  '4',
  '4',
  '0',
  '3',
  '*',
  5,
  5,
  '1',
  1,
  '1',
  '3',
  '2',
  3,
  '3',
  4,
  5,
  '1',
  '2',
  5,
  3,
  1,
  '5',
  5,
  5,
  '2',
  4,
  '3',
  4,
  '4',
  3,
  2,
  '1',
  5,
  4,
  '0',
  0,
  3,
  '5',
  '2',
  2,
  2],
 ['2',
  '0',
  0,
  '3',
  '3',
  '1',
  '3',
  '2',
  '0',
  '1',
  0,
  4,
  '4',
  1,
  '3',
  '2',
  '5',
  0,
  '*',
  4,
  5,
  '2',
  '0',
  3,
  2,
  4,
  '3',
  3,
  2,
  '3',
  4,
  '0',
  4,
  '1',
  5,
  1,
  '3',
  1,
  3,
  '1',
  0,
  5,
  '4',
  '5',
  5,
  1],
 ['4',
  '1',
  4,
  '1',
  '2',
  '0',
  '0',
  '5',
  '2',
  '2',
  2,
  3,
  '4',
  0,
  '2',
  '*',
  '0',
  0,
  '3',
  1,
  1,
  '4',
  '4',
  3,
  5,
  3,
  '5',
  3,
  2,
  '4',
  0,
  '5',
  1,
  '3',
  4,
  1,
  '5',
  4,
  4,
  '5',
  1,
  4,
  '2',
  '2',
  5,
  1],
 ['4',
  '4',
  3,
  '2',
  '0',
  '0',
  '0',
  '0',
  '*',
  '4',
  0,
  0,
  '0',
  3,
  '2',
  '0',
  '0',
  4,
  '5',
  4,
  0,
  '0',
  '2',
  0,
  4,
  4,
 

In [64]:
# mergerd_dfを組み合わせて全パターンのレコードを作る、Bbと比較して最も近いやつを採用する
mls = merged_df.values.tolist()
# print(mls)
# movie idに対する候補の値のリストを辞書で管理する
# もし候補に0があったらそれで確定しちゃってもいいかも
cand_value_dict = {}

col = 5
for id in MOVIE_IDS:
    cand_value_dict[id] = []
    for i in range(len(mls)):
        if not(np.isnan(mls[i][col])) and not(mls[i][col] in cand_value_dict[id]):
            cand_value_dict[id].append(int(mls[i][col]))
    col += 1
for key, value in cand_value_dict.items():
    if (0 in value) :#and (len(value) > 2):
        cand_value_dict[key] = [0]
print(cand_value_dict)

cand_val = list(cand_value_dict.values())
math.prod(len(lst) for lst in cand_val)

{'2': [4], '56': [0], '247': [4, 3, 2], '260': [3, 5, 2], '653': [0], '673': [0], '810': [0], '885': [3], '1009': [5], '1073': [0], '1097': [4], '1126': [2], '1525': [0], '1654': [1], '1702': [4], '1750': [0], '1881': [0], '1920': [5, 4, 3], '1967': [0], '2017': [0], '2021': [0], '2043': [0], '2086': [5], '2087': [0], '2093': [1], '2100': [1], '2105': [0], '2138': [0], '2143': [0], '2174': [2], '2193': [5], '2253': [5], '2399': [0], '2628': [0], '2797': [5], '2872': [5], '2968': [4], '3393': [0], '3438': [3], '3439': [0], '3440': [5], '3466': [0], '3479': [0], '3489': [3], '3877': [0], '3889': [1]}


27

In [65]:

# BaのN行目のGAOZに対応するratingを再構築しようとしてる
# C0^9を探索して値の候補を得たので、それを元にratingの組み合わせの候補を作成した
# その組み合わせのなかで、Bbに含まれるratingeとの最小のhammingが最も小さくなる組みを候補とする
prod = list(itertools.product(*list(cand_value_dict.values())))
res_ls = []
res_ham = 100
min_dif_ls = []
for cand_ls in  prod:
    min_dif = 100
    tmp_ls = []
    for rec_Bb in Bbls:
        difference = 0
        # hamming distance
        for i in range(len(cand_ls)):
            if rec_Bb[i] == '*':
                continue 
            if cand_ls[i] != int(rec_Bb[i]):
                difference += 1
        if difference < min_dif:
            min_dif = difference
    min_dif_ls.append(min_dif)
    if min_dif < res_ham:
        res_ham = min_dif
        res_ls = cand_ls
print(min_dif_ls)
res_ls
    

[6, 6, 5, 5, 5, 4, 6, 6, 5, 6, 6, 5, 5, 5, 4, 6, 6, 5, 6, 6, 5, 5, 5, 4, 6, 6, 5]


(4,
 0,
 4,
 5,
 0,
 0,
 0,
 3,
 5,
 0,
 4,
 2,
 0,
 1,
 4,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 5,
 0,
 1,
 1,
 0,
 0,
 0,
 2,
 5,
 5,
 0,
 0,
 5,
 5,
 4,
 0,
 3,
 0,
 5,
 0,
 0,
 3,
 0,
 1)

## 攻撃

### 以上の処理をBaの各行（0~49行）に対して実行する

In [66]:
import pandas as pd
import numpy as np

final_df = pd.DataFrame(columns=ALL_HEADER)

for row_index in range(50):
    # Baデータフレームから現在の行の情報を取得
    gender = Ba.iloc[row_index]["Gender"]
    age = int(Ba.iloc[row_index]["Age"])
    occupation = int(Ba.iloc[row_index]["Occupation"])
    zipcode = int(Ba.iloc[row_index]["ZIP-code"])

    filtered_C_list = []
    for c_index, cf in enumerate(c_data_list):
        min_hamming_distance = float("inf")
        cand = []
        # Cの10000レコードのそれぞれにハミング距離を計算
        for index, c_row in cf.iterrows():
            hamming_distance = GAOZ_weighted_hamming(c_row, gender, age, occupation, zipcode)
            if hamming_distance == min_hamming_distance:
                cand.append((c_row, hamming_distance))
            if hamming_distance < min_hamming_distance:
                min_hamming_distance = hamming_distance
                cand = [(c_row, hamming_distance)]
        # ハミング距離とCのファイルのindexを追加
        for elem, hamming_distance in cand:
            headers = copy.deepcopy(ALL_HEADER)
            headers.append("GAOZ_Hamming_Dist")
            headers.append("C_index")
            elem["GAOZ_Hamming_Dist"] = hamming_distance
            elem["C_index"] = c_index
            filtered_C_list.append(pd.Series(elem, index=headers))

    merged_df = pd.DataFrame(filtered_C_list)

    candidate_rows_df = pd.DataFrame(filtered_C_list)

    # 各列に対して、ランダムに値を選択（多数決の実装）
    elected_value = {}
    for column in ALL_HEADER:
        elected_value[column] = select_random_value(candidate_rows_df[column])

    # 選択された値から新しい行を作成
    elected_value_row_df = pd.DataFrame([elected_value])

    # 選択された行をfinal_dfの最後に追加
    final_df = pd.concat([final_df, elected_value_row_df], ignore_index=True)

final_df

KeyboardInterrupt: 

### 最終的に得た映画のレビューの候補とBbを比較し、Bbの何番目のindexが一番近くてハミング距離にどれくらいの違いがあるかを取得する

In [ ]:

def hamming_distance(a, b):
    a_int = a.fillna(0).astype(int)
    b_int = b.replace('*', None).fillna(0).astype(int)
    # ハミング距離の計算
    return np.sum(a_int != b_int)

def find_closest_row(row, Bb):
    distances = Bb.apply(lambda bb_row: hamming_distance(row[MOVIE_IDS], bb_row[MOVIE_IDS]), axis=1)
    min_distance = distances.min()
    closest_index = distances.idxmin()
    return pd.Series({'Min_Hamming_Distance': min_distance, 'Closest_Bb_Index': closest_index})

# 結果を格納するデータフレーム
result_df = pd.DataFrame(index=final_df.index, columns=['Min_Hamming_Distance', 'Closest_Bb_Index'])

# 各行に対してハミング距離を計算し、最も近い行を見つける
for idx, row in final_df[MOVIE_IDS].iterrows():
    result = find_closest_row(row, Bb[MOVIE_IDS])
    result_df.loc[idx] = result

# 元のfinal_dfにハミング距離の結果を結合
final_result_df = pd.concat([final_df, result_df], axis=1)
final_result_df

final_result_dfの分析

In [ ]:
# final resultの分析

hammings = final_result_df['Min_Hamming_Distance'].tolist()
# print(hammings)
dic = {
    '0-9': 0,
    '10-19':0,
    '20-29':0,
    '30-':0,
}
for num in hammings:
    if num < 10:
        dic['0-9'] += 1
    elif num < 20:
        dic['10-19'] += 1
    elif num < 30:
        dic['20-29'] += 1
    else:
        dic['30-'] += 1
print(dic)

攻撃結果の出力

In [ ]:
# 攻撃結果の出力
# index, rating